<a href="https://colab.research.google.com/github/jdasam/aat3020-2023/blob/main/notebooks/4_Machine_translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Translation

In [1]:
# install HuggingFace
!pip install transformers tokenizers

# If you are not using Colab, install below to read xlsx file
# !pip install openpyxl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 53.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 13.7 MB/s eta 0:00:00


In [3]:
'''
Download dataset (originally from NIA AI-Hub)
'''

# !gdown 1CpsqOuuuB3I_PG5DbuqH1ssCFVerU46g
# !unzip -q nia-aihub-korean-english.zip

# !gdown 1GMFWREWBVcD5vJdwNFadHmzVStclcyKf
# !unzip -q nia-aihub-korean-english-txt.zip

!gdown 13CGLEULYccogSLByHXPAxSveLZTtnj8c
!unzip -q nia_korean_english_csv.zip

Downloading...
From (original): https://drive.google.com/uc?id=13CGLEULYccogSLByHXPAxSveLZTtnj8c
From (redirected): https://drive.google.com/uc?id=13CGLEULYccogSLByHXPAxSveLZTtnj8c&confirm=t&uuid=85d68fb0-96a2-4844-980b-95f770c2cb90
To: /home/teo/userdata/git_libraries/aat3020/notebooks/nia_korean_english_csv.zip
100%|████████████████████████████████████████| 190M/190M [00:06<00:00, 28.9MB/s]


In [4]:
from pathlib import Path
import pandas as pd

df = pd.read_csv("nia_korean_english.csv")

In [ ]:
df

,원문,번역문
0,'Bible Coloring'은 성경의 아름다운 이야기를 체험 할 수 있는 컬러링 ...,Bible Coloring' is a coloring application that...
1,씨티은행에서 일하세요?,Do you work at a City bank?
2,푸리토의 베스트셀러는 해외에서 입소문만으로 4차 완판을 기록하였다.,"PURITO's bestseller, which recorded 4th rough ..."
3,11장에서는 예수님이 이번엔 나사로를 무덤에서 불러내어 죽은 자 가운데서 살리셨습니다.,In Chapter 11 Jesus called Lazarus from the to...
4,"6.5, 7, 8 사이즈가 몇 개나 더 재입고 될지 제게 알려주시면 감사하겠습니다.",I would feel grateful to know how many stocks ...
...,...,...
1602413,힐튼호텔 건너편에서 회현동주민센터로 내려가는 길에는 칙칙한 회색의 석축옹벽이 이어져...,"Across from the Hilton Hotel, the path to Hoeh..."
1602414,힘든 분들이 많이 계시지만 조금이나마 어렵지 않도록 도움이 될 수 있도록 저희가 최...,There are a lot of people who are having a har...
1602415,힘든 역사 속에서 대한민국을 불과 50여년 만에 빛나는 나라로 만들고 업적을 만들 ...,It was possible to make the Republic of Korea ...
1602416,힘든 일을 하는 데는 무엇보다 정부가 큰 관심을 갖고 있다는 자부심을 갖도록 해야 한다.,"As for doing hard work, most of all, they shou..."


In [ ]:
df['원문'][10000:10050], df['번역문'][10000:10050]

## Huggingface Tokenizer

In [ ]:
from tokenizers import BertWordPieceTokenizer
tokenizer = BertWordPieceTokenizer(strip_accents=False, lowercase=False)

num_files = len(data_list)

corpus_file   =  [str(path.parent / (path.stem + '_kor.txt')) for path in data_list[:num_files]]
# output_dir   = Path('hugging_kor_%d'%(vocab_size))
en_corpus_file   =  [str(path.parent / (path.stem + '_eng.txt')) for path in data_list[:num_files]]
# output_dir   = Path('hugging_eng_%d'%(vocab_size))

vocab_size    = 32000  # Number of maximum size of the vocabulary
limit_alphabet= 6000   
output_dir    = Path('hugging_kor_%d'%(vocab_size))
en_output_dir = Path('hugging_eng_%d'%(vocab_size))
output_dir.mkdir(exist_ok=True)
en_output_dir.mkdir(exist_ok=True)
min_frequency = 5 

tokenizer.train(files=corpus_file,
               vocab_size=vocab_size,
               min_frequency=min_frequency,
               limit_alphabet=limit_alphabet, 
               show_progress=True)

tokenizer.save_model(str(output_dir))

en_tokenizer = BertWordPieceTokenizer(strip_accents=False, lowercase=False)
en_tokenizer.train(files=en_corpus_file,
                vocab_size=vocab_size,
                min_frequency=min_frequency,
                limit_alphabet=limit_alphabet,
                show_progress=True)
en_tokenizer.save_model(str(en_output_dir))



['hugging_eng_32000/vocab.txt']

In [4]:
from transformers import BertTokenizerFast

tokenizer_src = BertTokenizerFast.from_pretrained('hugging_kor_32000',
                                                       strip_accents=False,
                                                       lowercase=False) 
tokenizer_tgt = BertTokenizerFast.from_pretrained('hugging_eng_32000',
                                                       strip_accents=False,
                                                       lowercase=False) 

tokenized_data = tokenizer_src(df['원문'].iloc[10])
print(df['원문'].iloc[10])
print(tokenizer_src.decode(tokenized_data['input_ids']))

나는 친구에게 그 철학자의 책을 선물해 주겠다고 말했습니다.
[CLS] 나는 친구에게 그 철학자의 책을 선물해 주겠다고 말했습니다. [SEP]


In [ ]:
tokenized_data['input_ids']

In [ ]:
tokenized_ids = tokenizer_src("나는 서강대학교에 다닙니다")['input_ids']
tokenized_ids

[2, 6481, 25257, 8055, 4330, 2537, 10839, 3]

In [ ]:
tokenizer_src.decode(tokenized_ids)

'[CLS] 나는 서강대학교에 다닙니다 [SEP]'

## Divide Train / Validate/ Test Set
- using `np.random.choice`
    - To always get the same random shuffling result, you have to use `np.random.seed()`

In [ ]:
df

,ID,지자체,원문,번역문
0,261811,경기도,"""경기도가 말산업 육성을 위해 총예산 245,193천원으로 2013년 경기도 용인시...","""The Gyeonggi provincial government announced ..."
1,409852,경기도,"""경기도가 주최하고 경기FTA활용지원센터와 코트라가 주관한 이번 시장개척단은 지난 ...","""Organized by Gyeonggi provincial government a..."
2,352671,경기도,"""경기도가 주최하고 경기도비정규직지원센터가 주관한 이번 교육은 공공 부문이 직·간접...","""Organized by Gyeonggi provincial government a..."
3,308191,경기도,"""경기도가 주최하고 사단법인 한국장애인복지시설협회 경기도협회(대표자:정권)에서 주관...","""Organized by Gyeonggi provincial government a..."
4,185424,경기도,"""경기도가 주최하고 인구보건복지협회 경기지회, 아이낳기좋은세상 경기운동본부가 주관하...","""Hosted by Gyeonggi provincial government, org..."
...,...,...,...,...
100082,73961,서울시 중구,힐튼호텔 건너편에서 회현동주민센터로 내려가는 길에는 칙칙한 회색의 석축옹벽이 이어져...,"Across from the Hilton Hotel, the path to Hoeh..."
100083,462950,경기도,힘든 분들이 많이 계시지만 조금이나마 어렵지 않도록 도움이 될 수 있도록 저희가 최...,There are a lot of people who are having a har...
100084,333006,경기도,힘든 역사 속에서 대한민국을 불과 50여년 만에 빛나는 나라로 만들고 업적을 만들 ...,It was possible to make the Republic of Korea ...
100085,454399,경기도,힘든 일을 하는 데는 무엇보다 정부가 큰 관심을 갖고 있다는 자부심을 갖도록 해야 한다.,"As for doing hard work, most of all, they shou..."


## Define Dataset
- Each datasample has to return source sentence and target sentence
- You need a Tokenizer to get the tokenized result


## Split Dataset
- using ``torch.utils.data.random_split(dataset)``

## Define Collate function
- After implementing Dataset, we have to declare DataLoader that groups several training samples as a single batch
- However, we cannot batchify the melodies in straightforward way, because the length of each melody is different
- In this problem, you will learn about how to handle sequences of different length as a batch


#### Pad Sequence and Pack Sequence
In PyTorch, there are two ways to batchify a group of sequence with different length.
- `torch.nn.utils.rnn.pad_sequence`
    - This function takes list of tensors with different length and return padded sequence
    - Padding is adding some constant number as a PAD token to match the length of short sequence to the maximum length
        - e.g. If there are sequence of length (3,7,4), we can add 4 zeros to sequence with length 3, 3 zeros to sequence with length 4 to make them length 7
    - In default, we use 0 for padding (zero padding)
    - The result 
- `torch.nn.utils.rnn.pack_sequence`
    - pad_sequence 

Cells below show the example of `pad_sequence`

1) However, the problem is that you can't figure out whether the 0 at the end of each sequence is a padded one, or was included in the original sequence
- e.g. `[2, 3, 4, 3, 0]` becomes `[ 2,  3,  4,  5,  0,  0,  0]`. Now we don't know how many zeros were added for padding

2) Also, if you run RNN for this padded sequence, RNN will calculate for the padded part also.
- RNN doesn't know whether it is padded data, or existing data
- This makes computation slower

3) If you want to use bi-directional, which also reads the sequence from backward, paddings can make the result different.

To solve this issue, we use PackedSequence, by using `pack_sequence`/

`PackedSequence` has `data` and `batch_sizes`
- `data` contains the flattened value of given batch
    - To optimize the computation, the sequences have to be sorted by descending of length
- `batch_sizes` represents how many valid batch sample exists for each time step
    - `[3, 3, 3, 2, 2, 1, 1]` means that there are 3 sequences for first three time steps, and then 2 sequences for next two steps, and then only 1 sequence for next two steps.
- `sorted_indices` shows how the sorted sequences can be converted to original order.
    - `[1,2,0]` means that 
        - the 0th sequence in the sorted sequences (the longest one) was indexed as 1 in the original input batch
        - the 1st sequence in the sorted sequences (`middle`) was indexed as 2 in the original input batch
        - the 2nd sequence in the sorted sequences (`short`) was index as 0 in the original input batch
- `unsorted_indices` shows how the original sequences are sorted.
    - `[2,0,1]` means that
        - the 0th sequence in the original input was sorted as 2nd in the sorted sequences

- RNN or its family of PyTorch can automatically handle `PackedSequence`
- However, other layers like `nn.Embedding` or `nn.Linear` cannot take `PackedSequence` as its input
- There are two ways to feed `PackedSequence` to these layers
    - First, convert PackedSequence to ordinary torch.Tensor by `torch.nn.utils.rnn.pad_packed_sequence`
        - This will convert PackedSequence to a tensor of sequneces with same length but different padding
    - The other way is to feed only PackedSequence.data, and then declaring new PackedSequence with the output as `data`

In [ ]:
'''
This will make error, because other layers cannot handle PackedSequence
'''
test_linear_layer = nn.Linear(in_features=1, out_features=2)
test_linear_layer(packed_sequence)

TypeError: ignored

In [ ]:
'''
One way to to this is using torch.nn.utils.rnn.pad_packed_sequence to convert PackedSequence to ordinary padded tensor
'''

from torch.nn.utils.rnn import pad_packed_sequence
padded_sequence, batch_lengths = pad_packed_sequence(packed_sequence)
print(f'The padded sequence generated from packed sequence (squeezed for printing): \n {padded_sequence.squeeze()}')
print(f'"pad_packed_sequence" also returns "batch_lengths", to clarify the original length before the padding: \n {batch_lengths}')



The padded sequence generated from packed sequence (squeezed for printing): 
 tensor([[ 3., 27., 15.],
        [ 2., 26., 14.],
        [ 1., 25., 13.],
        [ 0., 24., 12.],
        [ 0., 23., 11.],
        [ 0., 22., 10.],
        [ 0., 21.,  0.],
        [ 0., 20.,  0.]])
"pad_packed_sequence" also returns "batch_lengths", to clarify the original length before the padding: 
 tensor([4, 8, 6])


In [ ]:
'''
Now you can feed padded sequence to linear layer.
'''

linear_output = test_linear_layer(padded_sequence)
print(f"Output of feeding padded_sequence to a linear layer: {linear_output}")
print("Caution that it returns non-zero values for timestep with zero padding, because linear layer has a bias")

Output of feeding padded_sequence to a linear layer: tensor([[[ 3.3898,  1.8743],
         [24.5948, 12.2365],
         [13.9923,  7.0554]],

        [[ 2.5063,  1.4426],
         [23.7113, 11.8048],
         [13.1088,  6.6237]],

        [[ 1.6227,  1.0108],
         [22.8277, 11.3730],
         [12.2252,  6.1919]],

        [[ 0.7392,  0.5791],
         [21.9442, 10.9413],
         [11.3417,  5.7602]],

        [[ 0.7392,  0.5791],
         [21.0606, 10.5095],
         [10.4581,  5.3284]],

        [[ 0.7392,  0.5791],
         [20.1771, 10.0777],
         [ 9.5746,  4.8966]],

        [[ 0.7392,  0.5791],
         [19.2936,  9.6460],
         [ 0.7392,  0.5791]],

        [[ 0.7392,  0.5791],
         [18.4100,  9.2142],
         [ 0.7392,  0.5791]]], grad_fn=<ViewBackward0>)
Caution that it returns non-zero values for timestep with zero padding, because linear layer has a bias


In [ ]:
'''
You can make the output as a PackedSequence, by using torch.nn.utils.rnn.pack_padded_sequence
'''
from torch.nn.utils.rnn import pack_padded_sequence
re_packed_sequence = pack_padded_sequence(linear_output, batch_lengths, enforce_sorted=False)
re_packed_sequence

PackedSequence(data=tensor([[24.5948, 12.2365],
        [13.9923,  7.0554],
        [ 3.3898,  1.8743],
        [23.7113, 11.8048],
        [13.1088,  6.6237],
        [ 2.5063,  1.4426],
        [22.8277, 11.3730],
        [12.2252,  6.1919],
        [ 1.6227,  1.0108],
        [21.9442, 10.9413],
        [11.3417,  5.7602],
        [ 0.7392,  0.5791],
        [21.0606, 10.5095],
        [10.4581,  5.3284],
        [20.1771, 10.0777],
        [ 9.5746,  4.8966],
        [19.2936,  9.6460],
        [18.4100,  9.2142]], grad_fn=<PackPaddedSequenceBackward0>), batch_sizes=tensor([3, 3, 3, 3, 2, 2, 1, 1]), sorted_indices=tensor([1, 2, 0]), unsorted_indices=tensor([2, 0, 1]))

In [ ]:
'''
Another way to do it is using PackedSequence.data
'''

linear_out_pack = test_linear_layer(packed_sequence.data)
packed_sequence_after_linear = PackedSequence(linear_out_pack, packed_sequence.batch_sizes, packed_sequence.sorted_indices, packed_sequence.unsorted_indices)
packed_sequence_after_linear

PackedSequence(data=tensor([[24.5948, 12.2365],
        [13.9923,  7.0554],
        [ 3.3898,  1.8743],
        [23.7113, 11.8048],
        [13.1088,  6.6237],
        [ 2.5063,  1.4426],
        [22.8277, 11.3730],
        [12.2252,  6.1919],
        [ 1.6227,  1.0108],
        [21.9442, 10.9413],
        [11.3417,  5.7602],
        [ 0.7392,  0.5791],
        [21.0606, 10.5095],
        [10.4581,  5.3284],
        [20.1771, 10.0777],
        [ 9.5746,  4.8966],
        [19.2936,  9.6460],
        [18.4100,  9.2142]], grad_fn=<AddmmBackward0>), batch_sizes=tensor([3, 3, 3, 3, 2, 2, 1, 1]), sorted_indices=tensor([1, 2, 0]), unsorted_indices=tensor([2, 0, 1]))

In [5]:
from torch.nn.utils.rnn import pack_sequence, PackedSequence

def pack_collate(raw_batch):
  source, target, shifted_target = zip(*raw_batch)
  return pack_sequence(source, enforce_sorted=False), pack_sequence(target, enforce_sorted=False), pack_sequence(shifted_target, enforce_sorted=False)

train_loader = DataLoader(train_set, batch_size=32, collate_fn=pack_collate)
batch = next(iter(train_loader))
batch

NameError: ignored

In [ ]:
src, tgt, shifted_tgt = batch
src.sorted_indices, tgt.sorted_indices

(tensor([21, 17, 14,  5, 27,  8, 24, 28,  3,  9, 13, 29,  4, 16,  0, 19, 22,  2,
         10, 20, 25, 23, 31, 15,  7, 18, 11, 12, 26, 30,  1,  6]),
 tensor([21,  5, 24, 13,  3, 17,  8,  4, 14, 16, 28, 27, 29,  0, 19,  9,  2, 22,
         25, 20, 18, 23, 31, 11, 10,  7, 30, 15,  1, 12, 26,  6]))

## Define Model
![image](https://raw.githubusercontent.com/tensorflow/nmt/master/nmt/g3doc/img/seq2seq.jpg)

In [7]:
import torch.nn as nn
class Seq2seq(nn.Module):
  def __init__(self, enc_vocab, dec_vocab, hidden_size, num_layers=2):
    super().__init__()
    self.encoder = Encoder(enc_vocab, hidden_size, num_layers=num_layers)
    self.decoder = Decoder(dec_vocab, hidden_size, num_layers=num_layers)

  def forward(self, src, tgt):
    enc_out = self.encoder(src)
    # if isinstance(src, PackedSequence) and isinstance(tgt, PackedSequence):
    #   enc_out = enc_out[:, src.unsorted_indices][:, tgt.sorted_indices ]
    dec_out = self.decoder(tgt, enc_out)
    return dec_out 

class Encoder(nn.Module):
  def __init__(self, num_vocab, hidden_size, num_layers=2):
    super().__init__()
    self.emb = nn.Embedding(num_vocab, hidden_size)
    self.rnn = nn.GRU(hidden_size, hidden_size, batch_first=True, num_layers=num_layers)
    # batch_first True: it takes (Num_samples_in_batch, num_timesteps, num_dim)
    # batch_first False: it takes (num_timesteps, Num_samples_in_batch, num_dim)

  def forward(self, x):
    if isinstance(x, PackedSequence):
      emb = PackedSequence(self.emb(x.data), batch_sizes=x.batch_sizes, sorted_indices=x.sorted_indices, unsorted_indices=x.unsorted_indices)
    else:
      emb = self.emb(x)
    out, last_hidden = self.rnn(emb)
    # Even though input is PackedSequence, last_hidden is always tensor
    # in "unsorted_order"
    # last_hidden[:, 0] means 0th sample in the raw (unsorted) batch
    return last_hidden


class Decoder(nn.Module):
  def __init__(self, num_vocab, hidden_size, num_layers=2):
    super().__init__()
    self.emb = nn.Embedding(num_vocab, hidden_size)
    self.rnn = nn.GRU(hidden_size, hidden_size, batch_first=True, num_layers=num_layers)
    self.proj = nn.Linear(hidden_size, num_vocab)


  def forward(self, x, enc_output):
    if isinstance(x, PackedSequence):
      emb = PackedSequence(self.emb(x.data), batch_sizes=x.batch_sizes, sorted_indices=x.sorted_indices, unsorted_indices=x.unsorted_indices)
      out, last_hidden = self.rnn(emb, enc_output)
      # You have to feed last_hidden to rnn following unsorted_indices 
      # when you feed PackedSequence input to RNN.
      logit = PackedSequence(self.proj(out.data), batch_sizes=out.batch_sizes, sorted_indices=out.sorted_indices, unsorted_indices=out.unsorted_indices)
    else:
      emb = self.emb(x)
      out, last_hidden = self.rnn(emb, enc_output)
      logit = self.proj(out)
    return logit
    
hidden_size = 64
model = Seq2seq(tokenizer_src.vocab_size, tokenizer_tgt.vocab_size, hidden_size)
encoder = Encoder(tokenizer_src.vocab_size, hidden_size)
decoder = Decoder(tokenizer_tgt.vocab_size, hidden_size)

src, tgt, shifted_tgt = batch

enc_out = encoder(src)
dec_out = decoder(tgt, enc_out)

logit = model(src, tgt)
# logit.shape, logit.softmax(dim=-1)

NameError: ignored

In [ ]:
unsorted_enc_out = enc_out[:, src.unsorted_indices]
target_sorted_enc_out = unsorted_enc_out[:, tgt.sorted_indices]

In [ ]:
logit

PackedSequence(data=tensor([[-0.1037,  0.1473,  0.0048,  ..., -0.0047, -0.1110, -0.0725],
        [-0.1180,  0.1796,  0.0051,  ..., -0.0034, -0.1301, -0.0628],
        [-0.1426,  0.1752,  0.0121,  ..., -0.0245, -0.0783, -0.0516],
        ...,
        [ 0.0704,  0.0287,  0.0330,  ...,  0.0372, -0.1235,  0.0724],
        [ 0.0297,  0.0548, -0.0590,  ..., -0.0222, -0.1012,  0.0353],
        [ 0.0028,  0.0724, -0.0668,  ...,  0.0017, -0.0646,  0.0580]],
       grad_fn=<AddmmBackward0>), batch_sizes=tensor([8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 7, 7, 7, 7, 6, 5, 5, 5, 5, 5, 5, 5, 5, 5,
        5, 5, 5, 5, 5, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]), sorted_indices=tensor([5, 3, 4, 0, 2, 7, 1, 6]), unsorted_indices=tensor([3, 6, 4, 1, 2, 0, 7, 5]))

In [ ]:
logit.shape, shifted_tgt.shape, shifted_tgt

(torch.Size([1, 10, 30679]),
 torch.Size([1, 10]),
 tensor([[  76,   10,  746,  810,  906,  335,  412, 2590,   17,    3]]))

## Define Trainer

- You can download pre-trained weight

In [8]:
# !gdown 127OoPv8-lF5TWeNCq9PNa8igyGQnR-Pe
!gdown 15jL2TaRk6Q47uuPWDruUge6O_gCPv5mp

Downloading...
From: https://drive.google.com/uc?id=15jL2TaRk6Q47uuPWDruUge6O_gCPv5mp
To: /content/kor_eng_translator_model_vanilla_best.pt
100% 704M/704M [00:09<00:00, 78.0MB/s]


In [10]:
import torch

In [11]:
state_dict = torch.load("kor_eng_translator_model_vanilla_best.pt")

In [12]:
hidden_size = 512
model = Seq2seq(tokenizer_src.vocab_size, tokenizer_tgt.vocab_size, hidden_size, num_layers=3)

model.load_state_dict(state_dict['model'])

<All keys matched successfully>

In [15]:
class Translator:
  def __init__(self, model, src_tokenizer, tgt_tokenizer):
    self.model = model
    self.src_tokenizer = src_tokenizer
    self.tgt_tokenizer = tgt_tokenizer

    self.model.eval()
    self.model.cpu()

    self.start_token_idx = 2
    self.end_token_idx = 3

  def __call__(self, astr):
    encoded_tokens = self.src_tokenizer.encode(astr)
    encoded_tokens = torch.tensor(encoded_tokens, dtype=torch.long)

    # run models encoder to get encoder out
    enc_out = model.encoder(encoded_tokens)
    # run decoder one-by-one
    current_token = torch.tensor([self.start_token_idx]) # start of the sentence token
    prev_hidden = enc_out
    outputs = []
    for i in range(50):
      emb = self.model.decoder.emb(current_token)
      dec_out, cur_hidden = self.model.decoder.rnn(emb, prev_hidden)
      logit = self.model.decoder.proj(dec_out)
      selected_token_idx = torch.argmax(logit, dim=-1)

      # check the generated token is <end> token
      if selected_token_idx.squeeze().item() == self.end_token_idx:
        break

      # update information
      current_token = selected_token_idx
      prev_hidden = cur_hidden
      outputs.append(current_token.item())
    
    return self.tgt_tokenizer.decode(outputs)
  
translator = Translator(model, tokenizer_src, tokenizer_tgt)
translator('이 모델은 한국어 문장을 영어로 번역합니다')

'this model is translated into korean in english.'

In [ ]:
tokenizer_src.encode('나는 수업을 듣고 있습니다')

[2, 6481, 10499, 9414, 6507, 3]